In [1]:
from DB_System.communication import DB_Handle, DataHandle

In [2]:
dbhandler = DB_Handle(db_name='kospi_fin')
datahandler = DataHandle()

good


In [3]:
sql = f'''select * from 제주항공 '''

In [6]:
res = dbhandler.fetch(sql)
res = datahandler(res)